# Les arbres binaires

## Classe arbre

In [136]:
import random
import threading
import multiprocessing
import copy
import time

In [137]:
class My_thread(threading.Thread):
    def __init__(self,cle, noeud):
        threading.Thread.__init__(self)
        self.cle=cle
        self.arbre=Arbre(noeud)
    def run(self):
        return self.arbre.rechercher_cle(self.cle, self.arbre.noeud)
    def join(self):
        threading.Thread.join(self)
        return self.run()

In [161]:
class My_proc(multiprocessing.Process):
    def __init__(self,cle, noeud):
        multiprocessing.Process.__init__(self)
        self.cle=cle
        self.arbre=Arbre(noeud)
    def run(self):
        return self.arbre.rechercher_cle(self.cle, self.arbre.noeud)
    def join(self):
        multiprocessing.Process.join(self)
        return self.run()

In [191]:
class Arbre:
    def __init__(self,init):
        if type(init)==int:
            self.noeud={'cle':init, 'gauche':None, 'droite':None}
        else:
            self.noeud=copy.deepcopy(init)
    def generer_noeud(self, cle):
        noeud={'cle':cle, 'gauche':None, 'droite':None}
        return noeud
    def ajouter_noeud_aléa(self, nouveau_noeud, noeud=None):
        if noeud==None:
            noeud=self.noeud
        else:
            pass

        g_d=random.randint(0,1)
        if g_d==0:
            if noeud['gauche']==None:
                noeud['gauche']=nouveau_noeud
            else:
                self.ajouter_noeud_aléa(nouveau_noeud, noeud['gauche'])
        else:
            if noeud['droite']==None:
                noeud['droite']=nouveau_noeud
            else:
                self.ajouter_noeud_aléa(nouveau_noeud, noeud['droite'])

    def rechercher_cle(self, cle , noeud=None):
        exist = False
        if noeud == None:
            noeud = self.noeud
        else:
            pass
       
        if noeud['cle']==cle:
              return True
        else:
            if noeud["gauche"] != None:            
                if(noeud["gauche"]["cle"] != cle and exist != True):
                    exist = self.rechercher_cle(cle,noeud["gauche"])
                else:
                    return True                    
               
            if noeud["droite"] != None:
                if(noeud["droite"]["cle"] != cle and exist != True):
                    exist = self.rechercher_cle(cle,noeud["droite"])
                else:
                    return True          
        return exist
        
    def recherche_cle_non_opt( self,cle):
        return self.rechercher_cle(cle, self.noeud)
    def recherche_Cle_Concurente2( self,cle):
        noeud=self.noeud
        if noeud['cle']==cle:
            return True
        if noeud['gauche']!=None:
            thread1=My_thread(cle,noeud['gauche'])
            thread1.start()
        else:
            res_g=False
        if noeud['droite']!=None:
            thread2=My_thread(cle,noeud['droite'])
            thread2.start()
        else:
            res_d=False
        res_g=thread1.join()
        res_d=thread2.join()
        if res_g or res_d:
            return True
        else:
            return False
    def recherche_Cle_Concurente( self,cle):
        noeud=self.noeud
        list_noeud=[noeud]
        list_thread=[]
        i=0
        while i<len(list_noeud):
            thread=My_thread(cle,list_noeud[i])
            #thread=threading.Thread(target=self.rechercher_cle, args=(cle,list_noeud[i]))
            list_thread.append(thread)
            if list_noeud[i]['gauche']!=None:
                if list_noeud[i]['gauche']['gauche']!=None or list_noeud[i]['gauche']['droite']!=None:
                    list_noeud.append(list_noeud[i]['gauche'])
            if list_noeud[i]['droite']!=None:
                if list_noeud[i]['droite']['gauche']!=None or list_noeud[i]['droite']['droite']!=None:
                    list_noeud.append(list_noeud[i]['droite'])
            i=i+1
            [thread.start() for thread in list_thread]
            res=[thread.join() for thread in list_thread]
            return res[0]
    def recherche_Cle_Parallèle2( self,cle):
        noeud=self.noeud
        if noeud['cle']==cle:
            return True
        if noeud['gauche']!=None:
            proc1=My_proc(cle,noeud['gauche'])
            proc1.start()
        else:
            res_g=False
        if noeud['droite']!=None:
            proc2=My_proc(cle,noeud['droite'])
            proc2.start()
        else:
            res_d=False
        res_g=proc1.join()
        res_d=proc2.join()
        if res_g or res_d:
            return True
        else:
            return False
    def recherche_Cle_Parallèle( self,cle):
        noeud=self.noeud
        list_noeud=[noeud]
        list_proc=[]
        i=0
        while i<len(list_noeud):
            #proc=multiprocessing.Pool()
            #async_result = proc.apply_async(self.rechercher_cle, (cle,list_noeud[i]))
            #list_proc.append(async_result)
            proc=My_proc(cle,list_noeud[i])
            list_proc.append(proc)
            if list_noeud[i]['gauche']!=None:
                if list_noeud[i]['gauche']['gauche']!=None or list_noeud[i]['gauche']['droite']!=None:
                    list_noeud.append(list_noeud[i]['gauche'])
            if list_noeud[i]['droite']!=None:
                if list_noeud[i]['droite']['gauche']!=None or list_noeud[i]['droite']['droite']!=None:
                    list_noeud.append(list_noeud[i]['droite'])
            i=i+1
            #res=[async_result.get() for async_resul in list_proc]
            [proc.start() for proc in list_proc]
            res=[proc.join() for proc in list_proc]
            return res[0]
    def par_profondeur(self, noeud=None):
        if noeud==None:
            noeud=self.noeud
        list_cle=[]
        list_noeud=[]
        def to_list_profondeur(noeud):
            
            if len(list_noeud)>1 and noeud==self.noeud:
                return list_cle
            else:
                if noeud not in list_noeud:
                    list_cle.append(noeud['cle'])
                    list_noeud.append(noeud)

                if noeud['gauche']!=None:
                    return to_list_profondeur(noeud['gauche'])
                elif noeud['droite']!=None:
                    return to_list_profondeur(noeud['droite'])
                else:
                    noeud_=list_noeud[-2]
                    i=1
                    while (noeud_['droite']==None or noeud_['droite'] in list_noeud) and noeud_!=list_noeud[0]:
                        noeud_=list_noeud[-2-i]
                        i+=1
                    if noeud_['droite']!=None: 
                        if noeud_['droite'] not in list_noeud:
                            return to_list_profondeur(noeud_['droite'])
                        else:
                            return to_list_profondeur(noeud_)
                    else:
                        return to_list_profondeur(noeud_)

        to_list_profondeur(noeud)
        return list_cle
    def to_list_largeur(self, noeud=None):
        if noeud==None:
            noeud=self.noeud
        list_cle=[]
        list_noeud=[noeud]
        i=0
        while i<len(list_noeud):
            list_cle.append(list_noeud[i]['cle'])
            if list_noeud[i]['gauche']!=None:
                list_noeud.append(list_noeud[i]['gauche'])
            if list_noeud[i]['droite']!=None:
                list_noeud.append(list_noeud[i]['droite'])
            i=i+1
        return list_cle
    def afficher(self):
        print(self.noeud)
        

In [192]:
arb=Arbre(1)
nv=arb.generer_noeud(2)
arb.ajouter_noeud_aléa(nv)
nv1=arb.generer_noeud(4)
arb.ajouter_noeud_aléa(nv1)
nv2=arb.generer_noeud(3)
arb.ajouter_noeud_aléa(nv2)
nv3=arb.generer_noeud(5)
arb.ajouter_noeud_aléa(nv3)
nv4=arb.generer_noeud(6)
arb.ajouter_noeud_aléa(nv4)
nv5=arb.generer_noeud(7)
arb.ajouter_noeud_aléa(nv5)
arb.afficher()
arb2=Arbre(arb.noeud)
arb2.afficher()

{'cle': 1, 'gauche': {'cle': 4, 'gauche': None, 'droite': {'cle': 6, 'gauche': None, 'droite': None}}, 'droite': {'cle': 2, 'gauche': {'cle': 5, 'gauche': {'cle': 7, 'gauche': None, 'droite': None}, 'droite': None}, 'droite': {'cle': 3, 'gauche': None, 'droite': None}}}
{'cle': 1, 'gauche': {'cle': 4, 'gauche': None, 'droite': {'cle': 6, 'gauche': None, 'droite': None}}, 'droite': {'cle': 2, 'gauche': {'cle': 5, 'gauche': {'cle': 7, 'gauche': None, 'droite': None}, 'droite': None}, 'droite': {'cle': 3, 'gauche': None, 'droite': None}}}


In [193]:
debut= time.time()
print(arb.rechercher_cle(6))
fin=time.time()
print(fin-debut)

True
0.0004558563232421875


In [194]:
debut= time.time()
print(arb.recherche_Cle_Concurente(7))
fin=time.time()
print(fin-debut)

True
0.0015795230865478516


In [214]:
debut= time.time()
print(arb.recherche_Cle_Concurente2(8))
fin=time.time()
print(fin-debut)

False
0.004721641540527344


In [209]:
debut= time.time()
print(arb.recherche_Cle_Parallèle(8))
fin=time.time()
print(fin-debut)

False
0.012059688568115234


In [202]:
debut= time.time()
print(arb.recherche_Cle_Parallèle2(8))
fin=time.time()
print(fin-debut)

False
0.021582603454589844


In [215]:
arb.to_list_largeur()

[1, 4, 2, 6, 5, 3, 7]

In [216]:
arb.par_profondeur()

[1, 4, 6, 2, 5, 7, 3]